# Varroa Counter
## 1.  Définition du problème

Les colonies d'abeilles du monde entier sont infestées par un parasite qui s'appelle le varroa destructor.
Ce parasite au nom barbare se fixe sur le corps des abeilles adultes et se nourrit de l'hémolymphe. Les femelles pénètrent aussi dans les cellules operculées pour se reproduire sur les larves, ce qui crée plusieurs générations au sein d'une même cellule.
Le varroa transmet des virus aux abeilles et affaiblit leur système immunitaire.
Si rien n'est entrepris pour stopper leur prolifération, les colonies finissent par s'effondrer durant l'automne ou l'hiver.


<img src="https://github.com/geda/varroacounter/blob/main/varroa_destructor.jpg?raw=1" width="50%">

Une fois la récolte du miel effectuée, les apiculteurs effectuent différents traitements sur les colonies, notamment en utilisant de l'acide formique et de l'acide oxalique.
Une fois le traitement effectué, l'apiculteur dépose une planchette sous la ruche afin d'évaluer le degré d'infestation des colonies. Quelques jours après le traitement, les varroas morts tombent sur le fond de la ruche.
Les varroas ayant une taille de 1 à 2 mm, il devient très difficile de les compter lorsqu'ils sont nombreux. De plus, des résidus de cires d'abeilles tombent également des cadres, ce qui complique encore plus le comptage.

<img src="https://github.com/geda/varroacounter/blob/main/fond_varroas.jpg?raw=1" width="50%">

Les varroas sont les petites formes sombres allongées et arrondies.

## 2. Collecte de données
En recherchant des sets de données, j'en ai trouvé plusieurs disponibles sur https://universe.roboflow.com/, mais aucun dataset ne correspondait parfaitement à mes besoins:
* Images de varroas sur les abeilles et non sur la planche
* Images d'entraînement trop petites
* Images avec des varroas labellisés mais qui ne ressemblent pas vraiment à des varroas

J'ai donc créé un dataset avec mes propres images et j'en ai rajouté quelques-une trouvé sur le net. J'ai uploader le dataset sur roboflow sous le projet suivant: https://app.roboflow.com/varroa-counter/varroa-counter-v3/browse

### Inspection des données
Le dataset de données comprend des
* 32 images d'entraînement (71%)
* 13 images de validation (19%)
* 7 images de test (12%)

## 3. Préparation des Données
La taille maximal pour le redimmensionnent étant de 2048 sur Roboflow, j'ai redimensionné les images 1536 x 2048 pixels.

J'ai labellisé les 32 images en identifiant plus 1000 varroas. J'ai effectué ce travail très chronophage directement sur roboflow.


Chaque image de ce set a donc maintenant un label associé. Un seul nom de classe est utilisé pour ce dataset: **varroa**
Les labels associés à une image sont sauvegardés au format YOLO et comprennent simplement une suite de nombres comme ceci:
* 0 0.02587890625 0.25439453125 0.0107421875 0.0166015625
* 0 0.021484375 0.27880859375 0.009765625 0.0166015625
* 0 0.0751953125 0.3349609375 0.009765625 0.015625
* ...

#### Explication du format YOLO

```
0 0.02587890625 0.25439453125 0.0107421875 0.0166015625
│      │              │            │            │
│      │              │            │            └── height (hauteur normalisée de la bounding box)
│      │              │            └── width (largeur normalisée de la bounding box)
│      │              └── y_center (position Y du centre, normalisée)
│      └── x_center (position X du centre, normalisée)
└── class_id (identifiant de la classe = 0 = varroa)
```

| Valeur | Signification | Exemple |
|--------|---------------|---------|
| `0` | ID de la classe | varroa (seule classe du dataset) |
| `0.0259` | x_center | Le centre est à 2.6% de la largeur de l'image (très à gauche) |
| `0.2544` | y_center | Le centre est à 25.4% de la hauteur de l'image |
| `0.0107` | width | La box fait 1.07% de la largeur de l'image |
| `0.0166` | height | La box fait 1.66% de la hauteur de l'image |

## 4. Analyse Exploratoire des Données (AED)
Les données contiennent des images avec des fonds de différentes couleurs et matières.

## 5. Feature Engineering
Le modèle devra faire de la détection d'objets. Un des challenges sera de détecter des objets très petits dans les images.
Les 2 features importantes de la détection d'objets seront:
* Classification d'image: déterminer si des varroas sont présents dans l'image
* Localisation d'objet: trouver la position des varroas à l'aide de _bounding boxes_

## 6. Modélisation
Je choisi la classification comme modèle d'apprentissage.

Je divise mon ensemble de donnée comme ceci:
* 26 images d'entraînement (80%)
* 4 images de validation (12%)
* 2 images de test (8%)

In [ ]:
!pip install roboflow

import os
from roboflow import Roboflow
rf = Roboflow(api_key=str(os.getenv("ROBOFLOW_API_KEY")))

project = rf.workspace("varroa-counter").project("varroa-counter-large")
version = project.version(3)
dataset = version.download("yolov11")

loading Roboflow workspace...
loading Roboflow project...


## 7. Entraînement du modèle
### 📌 Version de Python recommandée

Pour cet exercice de réseaux de neurones convolutifs (CNN) avec YOLO11n, je vais utiliser **Python 3.12.7**, car c’est l’une des versions les mieux supportées par YOLO.

In [ ]:
# Vérifions la version de Python et du chemin de l'exécutable
import sys
print(sys.version)
print(sys.executable)

3.12.7 (tags/v3.12.7:0b05ead, Oct  1 2024, 03:06:41) [MSC v.1941 64 bit (AMD64)]
d:\dev\python\Python312\python.exe


In [ ]:
!pip install ultralytics

# import the needed librairies
import ultralytics

In [ ]:
from ultralytics import YOLO
# entrainement du modèle
# Je créer un nouveau modèle depuis zéro
model = YOLO('yolo11n.pt')

# lancement de l'entraînement avec seulement 1 seul passage et en abaissant le nombre de batch (nombre d'images traitées simultanément).
# le but est de tester si les capacitées de ma machine sont suffisantes pour entraîner mon modèle
results = model.train(data="varroa-counter-v3-2/data.yaml", epochs=1, imgsz=2048, batch=8,
                      max_det=2000, conf=0.1, iou = 0.5)
print (results)


New https://pypi.org/project/ultralytics/8.4.8 available  Update with 'pip install -U ultralytics'
Ultralytics 8.3.249  Python-3.12.7 torch-2.9.1+cpu CPU (Intel Core(TM) i7-10510U 1.80GHz)
engine\trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=16, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, compile=False, conf=0.1, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=varroa-counter-v3-2/data.yaml, degrees=0.0, deterministic=True, device=cpu, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=1, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=2048, int8=False, iou=0.5, keras=False, kobj=1.0, line_width=None, lr0=0.01, lrf=0.01, mask_ratio=4, max_det=2000, mixup=0.0, mode=train, model=yolo11n.pt, momentum=0.937, mosaic=1.0, multi_scale=False, name=train22, nbs=64, nms=False, opse

: 

Impossible d'entraîner mon modèle sur ma machine, car elle n'a pas de GPU et le kernel crash.
J'ai fait un essai en utilisant google colab, qui permet gratuitement d'entraîner des modèles avec un peu de GPU. Malheureusement j'ai le même problème sur google colab qui m'indique que ma session a planté après avoir utilisé toute la RAM disponible.

### Changement de stratégie: recherche d'un modèle existant
Je décide donc de rechercher un modèle existant pouvant couvrir mes besoins et j'ai trouvé le travail de recherche suivant: https://www.mdpi.com/2077-0472/15/9/969

**Référence:**
> Yániz, J.; Casalongue, M.; Martinez-de-Pison, F.J.; Silvestre, M.A.; Consortium, B.; Santolaria, P.; Divasón, J. *An AI-Based Open-Source Software for Varroa Mite Fall Analysis in Honeybee Colonies*. Agriculture 2025, 15, 969. https://doi.org/10.3390/agriculture15090969

Leurs modèle a été entraîné avec un dataset de 357 images sur plus de 500 epochs. Ils ont également livré un programme écrit en python permettant d'upload ses images et d'effectuer la détection des varroas avec leurs modèle.
Le code source est disponible sous github ainsi que leurs modèle entraîné: https://github.com/jodivaso/varrodetector/blob/main/model/weights/best.pt

En analysant le code j'ai trouvé particulièrement intéressant qu'ils utilisent une taille d'image assez grande de 6016 pixels (https://github.com/jodivaso/varrodetector/blob/main/varroa_mite_gui.py#L2507C42-L2507C46).



## 8. Evaluation du modèle
Je décide de rebalancer toutes les images de mon dataset en set de test et de n'appliquer aucun redimensionnement d'image

In [ ]:
from roboflow import Roboflow
rf = Roboflow(api_key="tEQkmVJiCxGOZMuDLR6d")
project = rf.workspace("varroa-counter").project("varroa-counter-v3")
version = project.version(3)
dataset = version.download("yolov11")

loading Roboflow workspace...
loading Roboflow project...



Extracting Dataset Version Zip to varroa-counter-v3-3 in yolov11:: 100%|██████████| 70/70 [00:00<00:00, 555.15it/s]


Il faut maintenant tester le modèle créer par l'étude espagnole avec mon dataset

In [ ]:
from ultralytics import YOLO

# Charger le modèle entraîné
model = YOLO('model_mdpi_3291496/weights/best.pt')

# Effectuer la détection sur l'image
results = model.val(
    data="varroa-counter-v3-2/data.yaml",
    split='val',  # or 'val' for validation set
    imgsz=3000,
    batch=8,
    conf=0.1,  # Seuil de confiance
    iou=0.5,
    max_det=2000
)

# Afficher les résultats
# Print results
print(f"Precision: {results.box.p}")
print(f"Recall: {results.box.r}")
print(f"mAP50: {results.box.map50}")
print(f"mAP50-95: {results.box.map}")

WARNING imgsz=[3000] must be multiple of max stride 32, updating to [3008]
Ultralytics 8.3.249  Python-3.12.7 torch-2.9.1+cpu CPU (Intel Core(TM) i7-10510U 1.80GHz)
Model summary (fused): 72 layers, 3,005,843 parameters, 0 gradients, 8.1 GFLOPs
val: Fast image access  (ping: 0.00.0 ms, read: 1561.7254.2 MB/s, size: 282.8 KB)
val: Scanning D:\dev\ia\cours\varroacounter\varroa-counter-v3-2\valid\labels.cache... 4 images, 0 backgrounds, 0 corrupt: 100% ━━━━━━━━━━━━ 4/4  0.0s


: 

In [ ]:
# Test du modèle avec le dataset Varroa-board-1
from ultralytics import YOLO

model = YOLO('model_mdpi_3291496/weights/best.pt')

results = model.val(
    data="Varroa-board-1/data.yaml",
    split='test',  # Utiliser le split 'test'
    imgsz=(6016), max_det=2000, conf=0.1, iou = 0.5,
    save=True, show_labels=False, line_width=2, save_txt=True, save_conf=True
)

print("\nRésultats de validation sur Varroa-board-1:")
print(f"Précision: {results.box.p}")
print(f"Recall: {results.box.r}")
print(f"mAP50: {results.box.map50}")
print(f"mAP50-95: {results.box.map}")

#Résultats de validation sur Varroa-board-1:
#Précision: [    0.80237]
#Recall: [    0.42177]
#mAP50: 0.584942355545071
#mAP50-95: 0.2293648037166341

Ultralytics 8.3.249  Python-3.12.7 torch-2.9.1+cpu CPU (Intel Core(TM) i7-10510U 1.80GHz)
Model summary (fused): 72 layers, 3,005,843 parameters, 0 gradients, 8.1 GFLOPs


FileNotFoundError: [34m[1mval: [0mError loading data from None
See https://docs.ultralytics.com/datasets for dataset formatting guidance.

In [ ]:
# Test avec découpage d'image en 4 parties
from ultralytics import YOLO
from PIL import Image
import os

# Charger le modèle
model = YOLO('model_mdpi_3291496/weights/best.pt')

# Chemin de l'image à tester
image_path = 'Varroa-board-1/test/images/IMG_0226_jpg.rf.c97161f83bb98300231bd6318d7dee3b.jpg'

# Charger l'image
img = Image.open(image_path)
width, height = img.size
print(f"Taille originale de l'image: {width}x{height}")

# Créer un dossier pour les images découpées
output_dir = 'temp_tiles'
os.makedirs(output_dir, exist_ok=True)

# Découper l'image en 4 parties (2x2)
half_width = width // 2
half_height = height // 2

tiles = []
positions = [
    (0, 0, half_width, half_height, "top_left"),
    (half_width, 0, width, half_height, "top_right"),
    (0, half_height, half_width, height, "bottom_left"),
    (half_width, half_height, width, height, "bottom_right")
]

# Découper et sauvegarder chaque partie
for i, (x1, y1, x2, y2, name) in enumerate(positions):
    tile = img.crop((x1, y1, x2, y2))
    tile_path = os.path.join(output_dir, f'tile_{i+1}_{name}.jpg')
    tile.save(tile_path)
    tiles.append((tile_path, name, x1, y1))
    print(f"Partie {i+1} ({name}): {tile.size[0]}x{tile.size[1]} sauvegardée")

print(f"\n{'='*60}")
print("DÉTECTION SUR CHAQUE PARTIE")
print(f"{'='*60}\n")

# Tester le modèle sur chaque partie
total_detections = 0
all_results = []

for tile_path, name, offset_x, offset_y in tiles:
    print(f"\n--- Détection sur {name} ---")
    results = model.predict(
        source=tile_path,
   imgsz=(6016), max_det=2000, conf=0.1, iou = 0.5,
    save=True, show_labels=False, line_width=2, save_txt=True, save_conf=True
    )

    detections = len(results[0].boxes)
    total_detections += detections
    all_results.append((name, detections, results[0]))

    print(f"Varroas détectés: {detections}")

print(f"\n{'='*60}")
print(f"RÉSUMÉ DES DÉTECTIONS")
print(f"{'='*60}")
print(f"\nNombre total de varroas détectés: {total_detections}")
print("\nDétail par partie:")
for name, count, _ in all_results:
    print(f"  - {name:15s}: {count:3d} varroas")

print(f"\nImages découpées sauvegardées dans: {output_dir}/")
print(f"Résultats de détection sauvegardés dans: runs/detect/predict*/")


Taille originale de l'image: 4284x5712
Partie 1 (top_left): 2142x2856 sauvegardée
Partie 2 (top_right): 2142x2856 sauvegardée
Partie 3 (bottom_left): 2142x2856 sauvegardée
Partie 4 (bottom_right): 2142x2856 sauvegardée

DÉTECTION SUR CHAQUE PARTIE


--- Détection sur top_left ---

image 1/1 d:\dev\ia\cours\varroacounter\temp_tiles\tile_1_top_left.jpg: 6016x4512 108 varroas, 7910.5ms
Speed: 315.9ms preprocess, 7910.5ms inference, 13.1ms postprocess per image at shape (1, 3, 6016, 4512)
Results saved to D:\dev\runs\detect\predict4
1 label saved to D:\dev\runs\detect\predict4\labels
Varroas détectés: 108

--- Détection sur top_right ---

image 1/1 d:\dev\ia\cours\varroacounter\temp_tiles\tile_2_top_right.jpg: 6016x4512 109 varroas, 6570.5ms
Speed: 272.7ms preprocess, 6570.5ms inference, 10.5ms postprocess per image at shape (1, 3, 6016, 4512)
Results saved to D:\dev\runs\detect\predict4
2 labels saved to D:\dev\runs\detect\predict4\labels
Varroas détectés: 109

--- Détection sur bottom_le